In [1]:
import gradio as gr
import sys
import logging
from dotenv import load_dotenv

# Reset và cấu hình logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ],
    force=True
)

load_dotenv()
sys.path.append('../backend')
sys.path.append('../backend/memory')

from chatbot_main import chatbot_response
from memory_manager import get_all_memories, add_memory, update_memory, delete_memory

/Users/lebaminhphuc/anaconda3/envs/llms/lib/python3.11/site-packages/google/rpc/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


2025-12-07 15:16:49,803 - INFO - HTTP Request: GET https://2f41421b-1095-4cb3-8625-6df79db78d48.us-east-1-1.aws.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
2025-12-07 15:16:50,623 - INFO - HTTP Request: GET https://2f41421b-1095-4cb3-8625-6df79db78d48.us-east-1-1.aws.cloud.qdrant.io:6333/collections/embedding_data "HTTP/1.1 200 OK"


In [2]:
def format_conversation_history(history):
    """
    Format conversation history từ Gradio thành string
    
    Args:
        history: List of tuples [(user_msg1, bot_msg1), (user_msg2, bot_msg2), ...]
                 hoặc None/empty list nếu không có lịch sử
    
    Returns:
        String format của conversation history
    """
    if not history or len(history) == 0:
        return ""
    
    formatted_history = []
    for i, (user_msg, bot_msg) in enumerate(history, 1):
        formatted_history.append(f"Lần {i}:")
        formatted_history.append(f"Người dùng: {user_msg}")
        formatted_history.append(f"Chatbot: {bot_msg}")
        formatted_history.append("")  # Dòng trống để phân cách
    
    return "\n".join(formatted_history)

def chat_interface(message, history):
    """
    Hàm xử lý chat với lịch sử hội thoại
    
    Args:
        message: Tin nhắn từ người dùng
        history: Lịch sử hội thoại (list of tuples)
    
    Returns:
        Phản hồi từ chatbot
    """
    try:
        # Format conversation history
        history_text = format_conversation_history(history)
        
        # Kết hợp lịch sử với message hiện tại
        if history_text:
            # Nếu có lịch sử, đưa vào user_query để backend có thể sử dụng
            enhanced_message = f"""Lịch sử hội thoại trước đó:
{history_text}

Câu hỏi hiện tại: {message}"""
        else:
            # Nếu không có lịch sử, chỉ dùng message
            enhanced_message = message
        
        # Gọi chatbot backend (logging sẽ tự động in ra từ chatbot_main.py)
        response = chatbot_response.invoke(enhanced_message)
        return response
    except Exception as e:
        logging.error(f"Error occurred: {str(e)}", exc_info=True)
        return f"Xin lỗi, đã xảy ra lỗi: {str(e)}"

In [3]:
# Custom CSS
custom_css = """
.message {
    font-size: 16px !important;
    line-height: 1.6 !important;
}
h1 {
    font-weight: 700 !important;
    letter-spacing: -0.02em !important;
    text-align: center !important;
}
.memory-display {
    font-family: monospace;
    white-space: pre-wrap;
}
"""

# Theme
theme = gr.themes.Default(
    primary_hue="blue",
    secondary_hue="sky",
    neutral_hue="slate",
    font=["Inter", "Segoe UI", "Roboto", "Helvetica Neue", "Arial", "sans-serif"],
    font_mono=["Fira Code", "Consolas", "Monaco", "monospace"]
).set(
    body_background_fill="white",
    body_text_color="#1f2937",
    button_primary_background_fill="#3b82f6",
    button_primary_text_color="white",
)

with gr.Blocks(theme=theme, css=custom_css) as demo:
    gr.Markdown("# 🏥 Chatbot Y tế")
    
    with gr.Tabs():
        # ========== TAB 1: CHAT ==========
        with gr.Tab("💬 Chat"):
            chatbot = gr.Chatbot(height=400)
            msg = gr.Textbox(
                placeholder="Nhập câu hỏi của bạn...",
                label="Tin nhắn",
                lines=2
            )
            with gr.Row():
                submit_btn = gr.Button("Gửi", variant="primary")
                clear_btn = gr.Button("Xóa hội thoại")
            
            gr.Examples(
                examples=[
                    "Thuốc Paracetamol có tác dụng gì?",
                    "Cách sử dụng thuốc giảm đau an toàn?",
                    "Tôi bị đau đầu nên uống thuốc gì?",
                ],
                inputs=msg
            )
            
            def respond(message, chat_history):
                bot_message = chat_interface(message, chat_history)
                chat_history.append((message, bot_message))
                return "", chat_history
            
            submit_btn.click(respond, [msg, chatbot], [msg, chatbot])
            msg.submit(respond, [msg, chatbot], [msg, chatbot])
            clear_btn.click(lambda: [], None, chatbot)
        
        # ========== TAB 2: MEMORY MANAGEMENT ==========
        with gr.Tab("🧠 Bộ nhớ dài hạn"):
            gr.Markdown("### Quản lý thông tin người dùng trong bộ nhớ dài hạn")
            
            # Hiển thị memories
            memory_display = gr.Textbox(
                label="📋 Danh sách thông tin hiện tại",
                lines=10,
                interactive=False,
                value=get_all_memories()
            )
            refresh_btn = gr.Button("🔄 Làm mới", variant="secondary")
            
            gr.Markdown("---")
            
            # Thêm memory mới
            gr.Markdown("#### ➕ Thêm thông tin mới")
            new_memory_input = gr.Textbox(
                label="Nhập thông tin mới",
                placeholder="Ví dụ: Người dùng bị dị ứng Penicillin",
                lines=2
            )
            add_btn = gr.Button("Thêm", variant="primary")
            
            gr.Markdown("---")
            
            # Sửa memory
            gr.Markdown("#### ✏️ Sửa thông tin")
            with gr.Row():
                edit_line_number = gr.Number(
                    label="Số dòng cần sửa",
                    value=1,
                    minimum=1,
                    precision=0
                )
                edit_content = gr.Textbox(
                    label="Nội dung mới",
                    placeholder="Nhập nội dung thay thế",
                    lines=2
                )
            edit_btn = gr.Button("Sửa", variant="primary")
            
            gr.Markdown("---")
            
            # Xóa memory
            gr.Markdown("#### 🗑️ Xóa thông tin")
            delete_line_number = gr.Number(
                label="Số dòng cần xóa",
                value=1,
                minimum=1,
                precision=0
            )
            delete_btn = gr.Button("Xóa", variant="stop")
            
            # Kết nối các button với function
            refresh_btn.click(fn=get_all_memories, outputs=memory_display)
            add_btn.click(fn=add_memory, inputs=new_memory_input, outputs=memory_display)
            edit_btn.click(fn=update_memory, inputs=[edit_line_number, edit_content], outputs=memory_display)
            delete_btn.click(fn=delete_memory, inputs=delete_line_number, outputs=memory_display)

/var/folders/hy/yvy_gmyj6r7cmgjg7md1rcxc0000gn/T/ipykernel_30968/3015401214.py:38: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=400)


2025-12-07 15:16:52,055 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


In [4]:
# Khởi chạy ứng dụng
if __name__ == "__main__":
    demo.launch(
        share=False,  # Đặt True nếu muốn chia sẻ link public
        show_error=True
    )


* Running on local URL:  http://127.0.0.1:7860
2025-12-07 15:16:51,362 - INFO - HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-12-07 15:16:51,369 - INFO - HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"
* To create a public link, set `share=True` in `launch()`.
